## 0- import Section 📚

In [1]:
#region imports
%load_ext autoreload 
%autoreload 2 
import pandas as pd
import random
from sklearn.neighbors import KNeighborsRegressor
from sklearn import set_config
set_config(display="diagram")

RANDOMSEED = 100
DISPLAY_WIDTH = 400
DISPLAYMAX_COLUMNS = 25

random.seed(RANDOMSEED)
pd.set_option('display.width', DISPLAY_WIDTH)
pd.set_option('display.max_columns', DISPLAYMAX_COLUMNS)
import warnings
warnings.filterwarnings('ignore')
warnings.filterwarnings(action='once')

#endregion

## 1- import data

In [6]:
# %pwd # to see thecurrent directory
# %cd '/home/ashamsa/code/pc/pde_cap_mrp_zagros'

In [7]:
from mrputils.loaders import DataLoader
from mrputils.processors import tweak_data,tweak_data4_prediction

dp=DataLoader("data")
dp.get_data() #loading all the data

df_awards=dp.df_awards
df_all_casting=dp.df_all_casting
df_all_details = dp.df_all_details


__,df_nulls=tweak_data(df_all_casting,df_all_details,df_awards)


starting class


/home/ashamsa/.pyenv/versions/3.8.12/envs/.pde_cap/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


## 2- defining X,y  📚

In [8]:
X=df_nulls.drop(columns=["id","revenue","popularity"])
X_with_id=df_nulls.drop(columns=["revenue","popularity"])
# X=df.drop(columns=["revenue","popularity"])
y=df_nulls[["revenue","popularity"]]
target_1=y.iloc[:,0]
target_2=y.iloc[:,1]


## 3- import all models


In [9]:
import pickle
M=pickle.load(open('models/finalized_model.sav',"rb")) # model for revenue
preprocessor=pickle.load(open('models/preprocessor_x.sav',"rb")) #processor #1
preprocessor_with_id=pickle.load(open('models/preprocessor_x_id.sav',"rb")) # processor #2
M_pop=pickle.load(open('models/popularity.sav',"rb"))#model popularity
knn_model=pickle.load(open('models/knn_similarity.sav',"rb"))

/tmp/ipykernel_29055/223644156.py:2: ResourceWarning: unclosed file <_io.BufferedReader name='models/finalized_model.sav'>
  M=pickle.load(open('models/finalized_model.sav',"rb")) # model for revenue
/tmp/ipykernel_29055/223644156.py:3: ResourceWarning: unclosed file <_io.BufferedReader name='models/preprocessor_x.sav'>
  preprocessor=pickle.load(open('models/preprocessor_x.sav',"rb")) #processor #1
/tmp/ipykernel_29055/223644156.py:4: ResourceWarning: unclosed file <_io.BufferedReader name='models/preprocessor_x_id.sav'>
  preprocessor_with_id=pickle.load(open('models/preprocessor_x_id.sav',"rb")) # processor #2
/tmp/ipykernel_29055/223644156.py:5: ResourceWarning: unclosed file <_io.BufferedReader name='models/popularity.sav'>
  M_pop=pickle.load(open('models/popularity.sav',"rb"))#model popularity
/tmp/ipykernel_29055/223644156.py:6: ResourceWarning: unclosed file <_io.BufferedReader name='models/knn_similarity.sav'>
  knn_model=pickle.load(open('models/knn_similarity.sav',"rb"))


## 4- Prediction

In [10]:
#sample record format for prediction; sample data called xx
id=597
record_data=df_all_details.query("id==@id")
record_casting=df_all_casting.query("id==@id")
xx=tweak_data4_prediction(df_all_casting,record_casting,df_all_details,record_data,df_awards)



/home/ashamsa/.pyenv/versions/3.8.12/envs/.pde_cap/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:524: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


In [11]:
M.predict(xx) #prediction

array([1.7235707e+09], dtype=float32)

## 14- Finding similar movies

In [13]:
num_neighbors=5 # Return the distances and index of the 2 closest points

In [12]:
xx_processed=preprocessor.transform(xx)#sample processed
X_with_id_processed=pd.DataFrame(preprocessor_with_id.transform(X_with_id),columns=preprocessor_with_id.get_feature_names_out())

In [14]:

similars=list(knn_model.kneighbors(xx_processed,n_neighbors=num_neighbors)[1][0])
similar_ids=list(X_with_id_processed.iloc[similars].remainder__id.values)
df_all_details.query('id  in @similar_ids')[["id","title","genres","release_date","revenue","popularity"]].merge(
    df_all_casting.query('id  in @similar_ids')[["id","director_name","actor1_name","actor2_name","actor3_name"]],on="id"
)


/home/ashamsa/.pyenv/versions/3.8.12/envs/.pde_cap/lib/python3.8/site-packages/sklearn/base.py:450: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


,id,title,genres,release_date,revenue,popularity,director_name,actor1_name,actor2_name,actor3_name
0,58,Pirates of the Caribbean: Dead Man's Chest,Adventure|Fantasy|Action,20/06/2006,1065659812,10.493411,Gore Verbinski,Johnny Depp,Orlando Bloom,Keira Knightley
1,597,Titanic,Drama|Romance|Thriller,18/11/1997,1845034188,10.517521,James Cameron,Kate Winslet,Leonardo DiCaprio,Frances Fisher
2,14161,2012,Action|Adventure|Science Fiction,10/10/2009,769653595,5.268971,Roland Emmerich,John Cusack,Amanda Peet,Chiwetel Ejiofor
3,37724,Skyfall,Action|Adventure|Thriller,25/10/2012,1108561013,9.818739,Sam Mendes,Daniel Craig,Judi Dench,Javier Bardem
4,102382,The Amazing Spider-Man 2,Action|Adventure|Fantasy,16/04/2014,705717432,6.966996,Marc Webb,Andrew Garfield,Emma Stone,Jamie Foxx


## 15- model predict popularity

In [15]:

M_pop.predict(xx) #prediction

array([9.8802595], dtype=float32)